In [ ]:
# ssh -L 8555:localhost:8555 username@hostanem

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
from lammps.calculator.client import LammpsDistributedClient
import zmq.asyncio
import asyncio
import time

MASTER_URI = "tcp://localhost:8556"
loop = zmq.asyncio.ZMQEventLoop()
asyncio.set_event_loop(loop)

DEBUG:asyncio:Using selector: ZMQSelector


In [3]:
client = LammpsDistributedClient(MASTER_URI, loop=loop)
loop.run_until_complete(client.create())

INFO:zmq_legos.mdp.client.Client:Connecting ZMQ socket to tcp://localhost:8556


In [4]:
script = """
            log  lammps.log
units  metal
dimension  3
boundary  p p p
atom_style  full
read_data  initial.data
kspace_style  pppm 0.000010
pair_style  buck/coul/long 10
pair_coeff 1 1 1309362.2766468062 0.104 0.0
pair_coeff 1 2 9892.357 0.20199 0.0
pair_coeff 2 2 2145.7345 0.3 30.2222
set type 2 charge -1.400000
set type 1 charge 1.400000
dump  1 all custom 1 mol.lammpstrj id type x y z fx fy fz
dump_modify  1 sort id
thermo_style  custom step etotal pxx pyy pzz pxy pxz pyz
run  0
"""

files = {
    'initial.data': """pymatgen autogenerated data file

8 atoms

2 atom types

0 4.1990858 xlo xhi
0 4.1990858 ylo yhi
0 4.1990858 zlo zhi
2.571198492157551e-16 2.571198492157551e-16 2.5711984921575517e-16 xy xz yz

Masses

1 24.305
2 15.9994

Atoms

1 1 1 1.4 0.0 0.0 0.0
2 1 1 1.4 3.3763224914577695e-16 2.0995429 2.0995429
3 1 1 1.4 2.0995429 0.0 2.0995429
4 1 1 1.4 2.0995429000000003 2.0995429 2.571198492157551e-16
5 1 2 -1.4 2.0995429000000003 2.0995429 2.0995429000000003
6 1 2 -1.4 2.0995429 0.0 1.2855992460787756e-16
7 1 2 -1.4 3.3763224914577695e-16 2.0995429 1.2855992460787756e-16
8 1 2 -1.4 0.0 0.0 2.0995429
"""
}

In [ ]:
# normal performance is about 250/second
async def run_lammps_job():
    # job = await workers.submit(script, files, properties={'stress', 'forces', 'energy'})
    # await job.future
    jobs = []
    start_time = time.perf_counter()
    for _ in range(5): 
        jobs.append(await client.submit(script, files, properties={'stress', 'forces', 'energy'}))
    results = await asyncio.gather(*jobs)
    print('time', 5, time.perf_counter() - start_time)
    return results

results = loop.run_until_complete(run_lammps_job())

print('===DONE===')

DEBUG:zmq_legos.mdp.client.Client:sending message to service b'lammps.job'
DEBUG:lammps.calculator.client.LammpsDistributedClient:lammps job 4a5bda00e2a84994bbc5903cbe49bee2 submitted
DEBUG:zmq_legos.mdp.client.Client:sending message to service b'lammps.job'
DEBUG:lammps.calculator.client.LammpsDistributedClient:lammps job 6c4a2425b0c346d9aa3bc67803bc3fd7 submitted
DEBUG:zmq_legos.mdp.client.Client:sending message to service b'lammps.job'
DEBUG:lammps.calculator.client.LammpsDistributedClient:lammps job becbab67e4434d809bc89b0f913b2970 submitted
DEBUG:zmq_legos.mdp.client.Client:sending message to service b'lammps.job'
DEBUG:lammps.calculator.client.LammpsDistributedClient:lammps job 057d73ff48944e78aac6719044ff35bc submitted
DEBUG:zmq_legos.mdp.client.Client:sending message to service b'lammps.job'
DEBUG:lammps.calculator.client.LammpsDistributedClient:lammps job 48d6fa51a3234a3db1e8f5632229eb59 submitted


In [1]:
# 121.9 1 core (121.95 per second)
# 244.4 2 cores (122.24 per second)
# 333.5 3 cores (111.18 per second)
# 378.7 4 cores (94.6 per second)

In [ ]:
# Edison Error
# 2017-10-31 12:48:42,866 - lammps.calculator.process.LammpsProcess - DEBUG - line: b'Tue Oct 31 12:47:35 2017: [unset]:_pmi_alps_get_apid:alps response not OKAY\n'
# 2017-10-31 12:48:42,867 - lammps.calculator.process.LammpsProcess - DEBUG - line: b'Tue Oct 31 12:47:35 2017: [unset]:_pmi_init:_pmi_alps_init returned -1\n'
# 2017-10-31 12:48:42,867 - lammps.calculator.process.LammpsProcess - DEBUG - line: b'[Tue Oct 31 12:47:35 2017] [c0-0c1s14n2] Fatal error in MPI_Init: Other MPI error, error stack:\n'
# 2017-10-31 12:48:42,867 - lammps.calculator.process.LammpsProcess - DEBUG - line: b'MPIR_Init_thread(537): \n'
# 2017-10-31 12:48:42,867 - lammps.calculator.process.LammpsProcess - DEBUG - line: b'MPID_Init(247).......: channel initialization failed\n'
# 2017-10-31 12:48:42,867 - lammps.calculator.process.LammpsProcess - DEBUG - line: b'MPID_Init(636).......:  PMI2 init failed: 1 \n'